In [7]:


import os, pandas as pd
# Add asset from file system
stateslatlong = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/statelatlong.csv')
print stateslatlong.head()

states = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/state.csv')
print states.head()


salesreps = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/salesrep.csv')
print salesreps.head()

        state   latitude   longitude
0     Alabama  32.806671  -86.791130
1      Alaska  61.370716 -152.404419
2     Arizona  33.729759 -111.431221
3    Arkansas  34.969704  -92.373123
4  California  36.116203 -119.681564
        state  expected revenue  customers initial_salesrep
0     Alabama            200000         10             Eric
1      Alaska            100000         12             Mark
2     Arizona            150000         15             Paul
3    Arkansas            150000          8             John
4  California            250000          6             John
  salesrep  index  capacity          home    quota
0     John      1        70    California  1000000
1     Jack      2        70      New York  1200000
2     Paul      3        70      Oklahoma   600000
3     Mark      4        50       Florida   800000
4     Eric      5        80  Pennsylvania  1400000


In [2]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

def distance( lat1, lon1, lat2, lon2 ):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance;

#print("Result:", distance(52.2296756, 21.0122287, 52.406374, 16.9251681))
#print("Should be:", 278.546, "km")

In [18]:
data = []
for s1 in states['state']:
    lat1 = stateslatlong.latitude[states.state == s1]
    lon1 = stateslatlong.longitude[states.state == s1]
    for s2 in states['state']:
        lat2 = stateslatlong.latitude[states.state == s2]
        lon2 = stateslatlong.longitude[states.state == s2]
        dist = distance(lat1, lon1, lat2, lon2)
        data.append([s1, s2, dist])
        
alldistances = pd.DataFrame(data=data, columns=['from', 'to', 'value'])
alldistances.head()

,from,to,value
0,Alabama,Alabama,0.000000
1,Alabama,Alaska,5569.338239
2,Alabama,Arizona,2288.410721
3,Alabama,Arkansas,568.679314
4,Alabama,California,3024.296944


In [19]:
alldistances.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/alldistance.csv', index=False)

In [4]:
data = []
for s1 in salesreps['salesrep']:
    home = salesreps.home[salesreps.salesrep == s1].iloc[0]
    lat1 = stateslatlong.latitude[states.state == home]
    lon1 = stateslatlong.longitude[states.state == home]
    for s2 in states['state']:
        lat2 = stateslatlong.latitude[states.state == s2]
        lon2 = stateslatlong.longitude[states.state == s2]
        dist = round(distance(lat1, lon1, lat2, lon2), 0)
        data.append([s1, s2, dist])
        
homedistances = pd.DataFrame(data=data, columns=['salesrep', 'state', 'value'])
homedistances.head()

,salesrep,state,value
0,John,Alabama,3024.0
1,John,Alaska,3620.0
2,John,Arizona,798.0
3,John,Arkansas,2467.0
4,John,California,0.0


In [5]:
homedistances.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/homedistance.csv', index=False)

In [11]:
data = []
for s1 in states['state']:
    prev = states.initial_salesrep[states.state == s1].iloc[0]
    for s2 in salesreps['salesrep']:
        disruption = 1
        if s2 == prev:
            disruption = 0
        data.append([s1, s2, disruption])
        
disruptions = pd.DataFrame(data=data, columns=['state', 'salesrep', 'value'])
disruptions.head()

,state,salesrep,value
0,Alabama,John,1
1,Alabama,Jack,1
2,Alabama,Paul,1
3,Alabama,Mark,1
4,Alabama,Eric,0


In [12]:
disruptions.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/disruption.csv', index=False)